In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.5 MB/s eta 0:00:00


In [ ]:
# ============================================================
# 0. Setup and Imports
# ============================================================
import os
import torch
import zipfile
from tqdm import tqdm
from pathlib import Path
from ultralytics import YOLO
from google.colab import drive

# Define device
device = 0 if torch.cuda.is_available() else 'cpu'

# ============================================================
# 1. Mount Google Drive
# ============================================================
print("="*60)
print("📂 MOUNTING GOOGLE DRIVE")
print("="*60)
drive.mount('/content/drive')
print("✅ Google Drive Mounted")

# ============================================================
# 2. Dataset Extraction (Method 3: Advanced)
# ============================================================
print("\n" + "="*60)
print("📦 DATASET EXTRACTION")
print("="*60)

# Path to zip (UPDATE THIS if your path changes)
ZIP_PATH = "/content/drive/MyDrive/yolo_dataset_1024.zip"
TARGET_DIR = '/content/'

if os.path.exists(ZIP_PATH):
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        members = zip_ref.namelist()
        print(f"📦 Extracting {len(members)} files from {Path(ZIP_PATH).name}...")
        for member in tqdm(members, desc="Extracting"):
            zip_ref.extract(member, TARGET_DIR)
    print("✅ Extraction Done!")
    # NOTE: The extraction should create a folder named 'yolo_dataset' inside /content/
else:
    print(f"❌ Zip not found: {ZIP_PATH}")
    print("💡 Please check the path and ensure Drive is mounted.")



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
📂 MOUNTING GOOGLE DRIVE
Mounted at /content/drive
✅ Google Drive Mounted

📦 DATASET EXTRACTION
📦 Extracting 16633 files from yolo_dataset_1024.zip...


Extracting: 100%|██████████| 16633/16633 [00:50<00:00, 326.83it/s]

✅ Extraction Done!


In [ ]:
# ============================================================
# 3. Create data.yaml file
# ============================================================
print("\n" + "="*60)
print("📄 CREATING data.yaml")
print("="*60)

YAML_FILE_PATH = '/content/data.yaml'

# Define the YAML content with correct structure and absolute paths
data_yaml_content = """
names:
- missing_hole
- mouse_bite
- open_circuit
- short
- spur
- spurious_copper
nc: 6
scale:
- 1.0
- 1.0
- 1.0
- 1.0
- 2.0
- 1.0
train: /content//train/images
val: /content//val/images
test: /content/test/images
"""

# Write the content to the file
if Path('/content/').exists():
    with open(YAML_FILE_PATH, 'w') as f:
        f.write(data_yaml_content)

    print(f"✅ data.yaml created at: {YAML_FILE_PATH}")
    print("\nContents:")
    print("------------------------------------------------------------")
    print(data_yaml_content.strip())
    print("--------------------------------------")
else:
    print(f"❌ Cannot create data.yaml. Base dataset folder '/content/yolo_dataset' not found.")
    print("💡 Did the zip file extract successfully?")




📄 CREATING data.yaml
✅ data.yaml created at: /content/data.yaml

Contents:
------------------------------------------------------------
names:
- missing_hole
- mouse_bite
- open_circuit
- short
- spur
- spurious_copper
nc: 6
scale:
- 1.0
- 1.0
- 1.0
- 1.0
- 2.0
- 1.0
train: /content//train/images
val: /content//val/images
test: /content/test/images
--------------------------------------


In [ ]:

# Clear cache
torch.cuda.empty_cache()

# ========================================================
# 2. PATHS (CHANGE ONLY THESE)
# ========================================================
YAML_FILE_PATH = '/content/data.yaml'   # Your data.yaml
PROJECT_DIR    = '/content/drive/MyDrive/PCB_Training'         # Where everything will be saved



In [ ]:
# ========================================================
# 3. LOAD MODEL
# ========================================================
print("Loading YOLOv8l (Teacher Model)...")
model = YOLO('yolov8l.pt')        # YOLOv8 large
# model = YOLO('yolov8x.pt')      # Uncomment for maximum accuracy
torch.cuda.empty_cache()
# ========================================================
# 4. TRAINING - FULLY COMPATIBLE WITH YOLOv8
# ========================================================
results = model.train(
    data=YAML_FILE_PATH,
    epochs=30,
    imgsz=512,
    batch=16,
    device=0,
    workers=8,
    cache='ram',        # Fast & stable
    amp=True,           # Mixed precision = faster + less VRAM

    # === BEST AUGMENTATIONS FOR PCB DEFECTS (YOLOv8 COMPATIBLE) ===
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    shear=2.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,         # Keep until last 10 epochs
    mixup=0.15,
    copy_paste=0.1,
    # close_up=True,    ← REMOVED (only in YOLOv11)

    # === OPTIMIZER & LR ===
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,           # Final LR = 0.001 × 0.01 = 1e-5
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,

    # === SAVING & LOGGING ===
    project=PROJECT_DIR,
    name='yolov8l_pcb_teacher_v8_compatible',
    exist_ok=True,
    patience=15,
    save_period=5,
    plots=True,
    save_json=True,
    save_hybrid=True,
    verbose=True
)

print("TRAINING COMPLETED SUCCESSFULLY!")

Loading YOLOv8l (Teacher Model)...
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in the future.
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.15, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8l_pcb_teacher_v8_compatible, nbs=64,

In [ ]:
# ========================================================
# 5. LOAD BEST WEIGHTS & FULL EVALUATION
# ========================================================
best_model_path = f"/content/drive/MyDrive/PCB_Training/yolov8l_pcb_teacher_v8_compatible/weights/best.pt"
model = YOLO(best_model_path)

print("\n" + "="*60)
print("EVALUATING ON VALIDATION SET")
print("="*60)
val_metrics = model.val(split='val', plots=True, save_json=True)

print("\n" + "="*60)
print("EVALUATING ON TEST SET")
print("="*60)
test_metrics = model.val(split='test', plots=True, save_json=True)

# ========================================================
# 6. PRINT OVERALL + CLASS-WISE RESULTS (BEAUTIFULLY)
# ========================================================
def print_classwise_results(metrics, title):
    print(f"\n{title}")
    print("-" * 80)
    print(f"{'Class':<15} {'Precision':>10} {'Recall':>10} {'mAP50':>10} {'mAP50-95':>12}")
    print("-" * 80)

    class_names = metrics.names
    for i, name in class_names.items():
        p = metrics.box.p[i]
        r = metrics.box.r[i]
        ap50 = metrics.box.ap50[i]
        ap = metrics.box.ap[i]
        print(f"{name:<15} {p:10.4f} {r:10.4f} {ap50:10.4f} {ap:12.4f}")

    print("-" * 80)
    print(f"{'OVERALL':<15} {metrics.box.mp:10.4f} {metrics.box.mr:10.4f} {metrics.box.map50:10.4f} {metrics.box.map:12.4f}")
    print(f"mAP50-95 (All): {metrics.box.map:.4f} | mAP50: {metrics.box.map50:.4f}")
    print(f"Precision (All): {metrics.box.mp:.4f} | Recall (All): {metrics.box.mr:.4f}")

# Print results
print_classwise_results(val_metrics, "VALIDATION SET - CLASS-WISE & OVERALL METRICS")
print_classwise_results(test_metrics, "TEST SET - CLASS-WISE & OVERALL METRICS")




EVALUATING ON VALIDATION SET
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 112 layers, 43,611,234 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 85.5±37.2 MB/s, size: 325.9 KB)
val: Scanning /content/val/labels.cache... 1123 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1123/1123 1.8Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 71/71 1.9it/s 36.5s
                   all       1123       4752      0.984      0.956      0.979      0.708
          missing_hole        187        810      0.992      0.998      0.993      0.773
            mouse_bite        186        787       0.97      0.934      0.961      0.681
          open_circuit        188        763      0.996      0.925      0.981      0.664
                 short        188        797      0.984      0.975      0.991      0.727
                  spur  

In [ ]:

# ========================================================
# 7. SAVE EVERYTHING NICELY
# ========================================================
import shutil
from datetime import datetime

final_folder = f"/content/drive/MyDrive/PCB_Final_Results_{datetime.now().strftime('%Y%m%d_%H%M')}"
os.makedirs(final_folder, exist_ok=True)

# Copy entire run
shutil.copytree(f"{PROJECT_DIR}/yolov8l_pcb_teacher_v8_compatible", final_folder, dirs_exist_ok=True)

print(f"\nALL RESULTS and  WEIGHTS and PLOTS SAVED TO:")
print(final_folder)



ALL RESULTS and  WEIGHTS and PLOTS SAVED TO:
/content/drive/MyDrive/PCB_Final_Results_20251125_1331


In [ ]:
%%javascript
function ClickConnect() {
  console.log("Keeping Colab alive...");
  document.querySelector("colab-toolbar-button#connect").click();
}
setInterval(ClickConnect, 60000);  // Click every 60 seconds

<IPython.core.display.Javascript object>